In [117]:
import h5py
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets
import torchvision.transforms as transforms

# FILE_PATH = "../AutoBots/h5_files/test_dataset.hdf5"
FILE_PATH = "../AutoBots/h5_files/train_dataset.hdf5"


hd5_file = h5py.File(FILE_PATH, "r")

In [118]:
print(list(hd5_file.keys()))
for key in hd5_file:
    print(f"KEY: {key}")
    print(hd5_file[key].shape)
    print(hd5_file[key].dtype)

['agents_trajectories', 'ego_trajectories', 'extras', 'orig_egos', 'road_pts']
KEY: agents_trajectories
(205942, 50, 15, 3)
float32
KEY: ego_trajectories
(205942, 50, 3)
float32
KEY: extras
(205942, 4)
float32
KEY: orig_egos
(205942, 50, 3)
float32
KEY: road_pts
(205942, 150, 10, 3)
float16


In [4]:
start = 0
duration = 10000
#N X 150 X 10 X 3
road_polyline = torch.Tensor(hd5_file["road_pts"][start:start+duration])

print(torch.all((torch.abs(road_polyline[:,:,:,2]) < 1e-6) | (torch.abs(road_polyline[:,:,:,2] - 1) < 1e-6)))

road_start_pts = road_polyline[:,:,1:,:2]
road_end_pts = road_polyline[:,:,:-1,:2]

print(road_start_pts.shape)
print(road_end_pts.shape)

tensor(True)
torch.Size([10000, 150, 9, 2])
torch.Size([10000, 150, 9, 2])


In [21]:
start = 0
duration = 10000
# N, 15, 20, 3
# N, agents, pts, xyz
raw_agent_trajectory = torch.Tensor(hd5_file["agents_trajectories"][start:start+duration])
raw_agent_trajectory = torch.movedim(raw_agent_trajectory, 1, 2)

is_zero_mask = torch.abs(raw_agent_trajectory)<1e-6
agent_present_mask = ~(torch.all(torch.flatten(is_zero_mask, 2,3), dim=-1))
agent_polyline_counts = torch.sum(agent_present_mask, dim=-1)
# print(torch.max(agent_counts), torch.min(agent_counts), torch.mean(agent_counts.float()))
print(agent_present_mask.shape)

road_start_pts = raw_agent_trajectory[:,:,1:,:2]
road_end_pts = raw_agent_trajectory[:,:,:-1,:2]

print(road_start_pts.shape)
print(road_end_pts.shape)

torch.Size([10000, 15])
torch.Size([10000, 15, 19, 2])
torch.Size([10000, 15, 19, 2])


In [116]:
from vectornet_argoverse_dataset import *

dataset = ArgoverseVectornetDataset(FILE_PATH)
# # for i in range(1):
# dataset[1]

loader = DataLoader(dataset, 64, shuffle=True, num_workers=8)
import time
device = torch.device("cpu")
print(len(dataset))
for epoch in range(10):
    print(epoch)
    n = 0
    start = time.time()
    for i, (x,y) in enumerate(loader):
        if n % 100 == 0:
            print(n/(len(loader)/64)*100)
        n += 1
    print(time.time()-start)
    print(n)

78143
0
0.0
524.1605241605241
1048.3210483210482
1572.4815724815726
2096.6420966420965
2620.802620802621
3144.963144963145
3669.123669123669
4193.284193284193
4717.444717444718
5241.605241605242
5765.7657657657655
6289.92628992629
25.534120321273804
1221
1
0.0
524.1605241605241
1048.3210483210482
1572.4815724815726
2096.6420966420965
2620.802620802621


KeyboardInterrupt: 